# parsing published_at

In [ ]:
for s in ["3 days ago", "2 weeks ago", "1 month ago", "13 minutes ago", "4 hours ago"]:
    print(s, "->", mvf.parse_date(s))

# main old 1

In [ ]:
import os
import re
import pandas as pd
import logging
import time
import json

from datetime import datetime, timedelta
from dotenv import load_dotenv
from pathlib import Path
from tqdm import tqdm
from collections import Counter
from typing import Optional

from openai import OpenAI
from difflib import get_close_matches
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

import func_bit as mvf  # Project-specific functions
import importlib
importlib.reload(mvf)

# Загрузка переменных окружения из .env
load_dotenv()

# Получение API-ключа
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

# Загрузка переменных и словарей
from config_bit import (
    DATA_FOLDER, REPORT_FOLDER, RESULT_FOLDER, 
    SCRAPING_DATE, DEDUPLICATION_MODE, KEEP,
    EMPLOYMENT_TYPES, WORK_TYPES,
    HARD_SKILLS, SOFT_SKILLS, LANG_PATTERNS, LANG_LEVELS,LANG_LEVEL_DESCRIPTIONS,
    GRADE_KEYWORDS, DIRECTION_PRIORITY, AI_SPETIALIST_TERMS,
    CURRENCY_RATES, PERIOD_MULTIPLIERS
)

# Создаем папки, если их нет
REPORT_FOLDER.mkdir(exist_ok=True)
RESULT_FOLDER.mkdir(exist_ok=True)

# Установка опций просмотра
pd.set_option("display.max_rows", 20) 
pd.set_option("display.max_columns", 50) 


# ========================== 1. Загружаем Excel напрямую в DataFrame ===
file_path = os.path.join("data", "dataset_linkedin-jobs-scraper-no-login-required_2025-12-14_09-12-49-010.xlsx")
df = pd.read_excel(file_path)

print("Начальное количество записей:", len(df))

# ========================== 2. Удаляем записи старше 6 месяцев ===
# дата отсечения: 6 месяцев назад от даты скрапинга
six_months_ago = SCRAPING_DATE - timedelta(days=180)

# получаем реальную дату публикации
df["posted_at"] = df["published_at"].apply(mvf.parse_date)

# фильтруем только свежие записи > 6 мес
df = df[df["posted_at"].notna() & (df["posted_at"] >= six_months_ago)]
print("После удаления старше 6 месяцев:", len(df))


# ========================== 3. Удаляем полные дубликаты ===
df = df.drop_duplicates()
print("После удаления полных дубликатов:", len(df))

# ========================== 4. Удаляем дубликаты по ключам job_title+company_name+location ===
df = df.drop_duplicates(
     subset=["job_title", "company_name", "location"],
     keep="first"
)
print("После удаления дубликатов по ключам:", len(df))

# ========================== 5. Убираем строки со значениями ["Mid-Senior level", "Not Applicable" в "seniority_level"
df_clean = df[~df["seniority_level"].isin(["Mid-Senior level", "Not Applicable"])]
print("Количество строк entry level после фильтрации:", len(df_clean))

# ========================== 6. чистим job_title в job_title_clean
df_clean = df_clean.copy()
# применяем функцию очистки к колонке job_title
df_clean.loc[:, "job_title_clean"] = mvf.clean_job_titles(df_clean["job_title"].tolist())

# ========================== 7. переименуем колонки для дальнейшего использования готовых функций
df_clean = df_clean.rename(columns={
    'company_name': 'company',
    'description_text': 'description',
    'employment_type' : 'contractType',
    'seniority_level' : 'experienceLevel'
})

display(df_clean)

# ========================== 8. Сохраняем результат в Excel preclean.xlsx ===
output_path = "preclean.xlsx"
df_clean.to_excel(output_path, index=False)
print(f"Файл сохранён: {output_path}")
print("Количество строк в итоговом датафрейме:", len(df_clean))

# # ========================== 9. Добавляем новый столбец с классификацией
# df = df_clean.copy()
# df.loc[:, "direction"] = df["description_text"].apply(
#     lambda x: mvf.classify_direction(x, AI_SPETIALIST_TERMS)
# )

# # Проверим распределение по направлениям
# print("-----------------------------")
# print(df["direction"].value_counts())

# ========================== 8 Извлечение при помощи AI из description directions, hard_skills, soft_skills и salary
# tqdm.pandas(desc="Анализ description_text...")
# df_clean[["directions_AI", "hard_skills", "soft_skills", "salary_range"]] = df_clean.progress_apply(
#     lambda row: pd.Series(
#         mvf.analyze_description(row["description_text"], row["salary_range"], client)
#     ),
#     axis=1
# )


# print(df_clean["direction"].value_counts())
#print(df_clean[["job_title", "hard_skills", "soft_skills", "salary_range"]])

## ========================== 9. Приведение ЗП к евро в год
# df_salary = mvf.normalize_salary_field(df_clean, salary_col="salary_range")

## ========================== 10. Нормализация locations - выделение города, земли, страны
# df_pre = mvf.clean_and_enrich_germany_locations(df_salary)

# # ========================== 11.. Сохранение результата pre-cleaned
# df_pre_path = RESULT_FOLDER / "03_merged_jobs_pre-clean.xlsx"
# df_pre.to_excel(df_pre_path, index=False)
# print(f"Информация после предварительного обогащения (salary, location) сохранена в файл: {df_pre_path}")

## ========================== 12. Запуск финального пайплайна на df_pre
# #reports = mvf.run_ecommerce_pipeline(df_pre, verbose=True, logger=logger)


## ========================== 13. Запуск просмотра
# #mvf.show_reports(reports, top_n=10)


# # 13 Анализ по федеральным землям
# # df_state_stats = mvf.analyze_jobs_by_state(df_clean, top_n=10)
# # print(df_state_stats)



# main old 2

In [ ]:
import os
import re
import pandas as pd
import logging
import time
import json

from datetime import datetime, timedelta
from dotenv import load_dotenv
from pathlib import Path
from tqdm import tqdm
from collections import Counter
from typing import Optional

from openai import OpenAI
from difflib import get_close_matches
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

import func_bit as mvf  # Project-specific functions
import importlib
importlib.reload(mvf)

# Загрузка переменных окружения из .env
load_dotenv()

# Получение API-ключа
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

# Загрузка переменных и словарей
from config_bit import (
    DATA_FOLDER, REPORT_FOLDER, RESULT_FOLDER, 
    SCRAPING_DATE, DEDUPLICATION_MODE, KEEP,
    EMPLOYMENT_TYPES, WORK_TYPES,
    HARD_SKILLS, SOFT_SKILLS, LANG_PATTERNS, LANG_LEVELS,LANG_LEVEL_DESCRIPTIONS,
    GRADE_KEYWORDS, DIRECTION_PRIORITY, AI_SPETIALIST_TERMS,
    CURRENCY_RATES, PERIOD_MULTIPLIERS
)

# Создаем папки, если их нет
REPORT_FOLDER.mkdir(exist_ok=True)
RESULT_FOLDER.mkdir(exist_ok=True)

# Установка опций просмотра
pd.set_option("display.max_rows", 20) 
pd.set_option("display.max_columns", 50) 


# ========================== 1. Загружаем Excel напрямую в DataFrame ===
file_path = os.path.join("data", "dataset_linkedin-jobs-scraper-no-login-required_2025-12-14_09-12-49-010.xlsx")
df = pd.read_excel(file_path)

print("Начальное количество записей:", len(df))

# ========================== 2. Удаляем записи старше 6 месяцев ===
# дата отсечения: 6 месяцев назад от даты скрапинга
six_months_ago = SCRAPING_DATE - timedelta(days=180)

# получаем реальную дату публикации
df["posted_at"] = df["published_at"].apply(mvf.parse_date)

# фильтруем только свежие записи > 6 мес
df = df[df["posted_at"].notna() & (df["posted_at"] >= six_months_ago)]
print("После удаления старше 6 месяцев:", len(df))


# ========================== 3. Удаляем полные дубликаты ===
df = df.drop_duplicates()
print("После удаления полных дубликатов:", len(df))

# ========================== 4. Удаляем дубликаты по ключам job_title+company_name+location ===
df = df.drop_duplicates(
     subset=["job_title", "company_name", "location"],
     keep="first"
)
print("После удаления дубликатов по ключам:", len(df))

# ========================== 5. Убираем строки со значениями ["Mid-Senior level", "Not Applicable" в "seniority_level"
EXCLUDE_LEVELS = ["Mid-Senior level", "Not Applicable"]
df_clean = df[~df["seniority_level"].isin(EXCLUDE_LEVELS)].copy()
print("Количество строк entry level после фильтрации:", len(df_clean))

# ========================== 6. чистим job_title в job_title_clean
df_clean = df_clean.copy()
# применяем функцию очистки к колонке job_title
df_clean.loc[:, "job_title_clean"] = mvf.clean_job_titles(df_clean["job_title"].tolist())

# ========================== 7. переименуем колонки для дальнейшего использования готовых функций
df_clean = df_clean.rename(columns={
    'company_name': 'company',
    'description_text': 'description',
    'employment_type' : 'contractType',
    'seniority_level' : 'experienceLevel'
})


#display(df_clean)

# # ========================= 8. Сохраняем результат в Excel preclean.xlsx ===
df_clean_path = RESULT_FOLDER / "clean_vacancies_res.xlsx"
df_clean.to_excel(df_clean_path, index=False)
print(f"Результирующий файл сохранён: {output_path}")


## HARD и SOFT скилы для джунов

In [ ]:
import pandas as pd
import plotly.express as px
from collections import Counter
from pathlib import Path

def plot_junior_skills_interactive(hard_counter, soft_counter, scraping_date, report_folder: Path, syst_name: str):
    """Строит одну диаграмму с переключателем HARD/SOFT skills и сохраняет в REPORT_FOLDER."""

    # преобразуем в DataFrame топ-10
    hard_df = pd.DataFrame(hard_counter.most_common(10), columns=["skill", "count"])
    soft_df = pd.DataFrame(soft_counter.most_common(10), columns=["skill", "count"])

    # базовый график (по умолчанию HARD)
    fig = px.bar(
        hard_df,
        x="skill",
        y="count",
        text="count",
        title=f"ТОП HARD skills джунов ({syst_name}, за пол года от {scraping_date.strftime('%d.%m.%Y')})",
        labels={"skill": "Навык", "count": "Количество"},
        color_discrete_sequence=["#4472C4"]
    )
    fig.update_traces(textposition="outside")

    # кнопки переключения
    buttons = [
        dict(
            label="HARD skills",
            method="update",
            args=[
                {"x": [hard_df["skill"]], "y": [hard_df["count"]], "text": [hard_df["count"]],
                 "marker": {"color": "#4472C4"}, "type": "bar"},
                {"title": {"text": f"ТОП HARD-skills джунов ({syst_name},  за пол года от {scraping_date.strftime('%d.%m.%Y')})",
                           "font": dict(family="Arial Bold, sans-serif", size=16, color="darkblue")}}
            ]
        ),
        dict(
            label="SOFT skills",
            method="update",
            args=[
                {"x": [soft_df["skill"]], "y": [soft_df["count"]], "text": [soft_df["count"]],
                 "marker": {"color": "#70AD47"}, "type": "bar"},
                {"title": {"text": f"ТОП SOFT-skills джунов ({syst_name}, за пол года от {scraping_date.strftime('%d.%m.%Y')})",
                           "font": dict(family="Arial Bold, sans-serif", size=16, color="darkgreen")}}
            ]
        )
    ]

    fig.update_layout(
        updatemenus=[dict(
            type="buttons",
            buttons=buttons,
            direction="down",  
            showactive=True,
            x=1.05,
            y=1.0,
            xanchor="left",
            yanchor="top"
        )],
        xaxis_tickangle=-30
    )


    # сохранение
    # output_path = report_folder / "junior_skills_interactive.html"
    # fig.write_html(output_path, auto_open=True)
    # print(f"Интерактивная диаграмма HARD/SOFT skills сохранена: {output_path}")
    fig.show()
